https://github.com/fastai/courses/blob/master/deeplearning1/nbs/char-rnn.ipynb

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%matplotlib inline
import importlib
import matplotlib.pyplot as plt
import matplotlib.ticker
import numpy as np
import random
import time
import os
import pandas as pd
import csv
import math
import bcolz
import pickle
import re
import pathlib
import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

SOS_TOKEN = '<SOS>' # Start Of Word

# Data setup

In [3]:
# importing keras might cause problems with cudann version etc
# import keras # some good utils in here
# path = keras.utils.data_utils.get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")

In [4]:
path = '/home/ohu/.keras/datasets/nietzsche.txt'
text = open(path).read().lower()
path
print('corpus length:', len(text))

'/home/ohu/.keras/datasets/nietzsche.txt'

corpus length: 600893


In [5]:
!tail {path} -n5

not to be regarded as a pure type. Knowledge and science--as far as they
existed--and superiority to the rest of mankind by logical discipline
and training of the intellectual powers were insisted upon by the
Buddhists as essential to sanctity, just as they were denounced by the
christian world as the indications of sinfulness.

In [6]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
''.join(chars)

total chars: 57


'\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxyzäæéë'

In [7]:
char_indices = dict((c, i) for i, c in enumerate(chars))
char_indices['\n']

0

# Utils

In [8]:
def string_to_tensor(in_str, chars_index=char_indices, as_variable=True):
    """Onehot encoded tensor of string"""
    tensor_length = len(in_str)
    tensor = torch.zeros(1, tensor_length, len(chars_index))
    for li, letter in enumerate(in_str):
        tensor[0, li, chars_index[letter]] = 1
    if as_variable:
        tensor = Variable(tensor).cuda()
    return tensor

string_to_tensor('hello', as_variable=False).size()

torch.Size([1, 5, 57])

In [9]:
def time_since(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

# Char Rnn

In [10]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(CharRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.layers = 1
        
        self.rnn = nn.GRU(input_size, hidden_size, batch_first=True)
        self.lin_output = nn.Linear(hidden_size, input_size)
        
        
    def forward(self, input_char_vs, hidden = None):
        if hidden is None:
            hidden = self.init_hidden()
        
        rnn_outputs, hidden = self.rnn(input_char_vs, hidden)
        
        output = self.lin_output(hidden)
        output = F.log_softmax(output[0])
       
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.layers, 1, self.hidden_size)).cuda()
    
model = CharRNN(len(chars), 512).cuda()
model

CharRNN (
  (rnn): GRU(57, 512, batch_first=True)
  (lin_output): Linear (512 -> 57)
)

In [11]:
#tmp = Variable(torch.FloatTensor([2]).view(1,1,-1)).cuda()
#tmp = Variable(string_to_tensor('hello')).cuda()
tmp = model(string_to_tensor('hello'))
len(tmp[0][0])
tmp[1].size()
#chars[tmp[0].topk(1)[1].data[0][0]]

57

torch.Size([1, 1, 512])

In [12]:
tmp[0][0].topk(1)[1].data[0]

29

# Eval

In [13]:
def print_example(iters=320, choice=True):
    seed_string="ethics is a basic foundation of all that"
    for i in range(iters):
        #x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        x = string_to_tensor(seed_string)
        output, hidden = model(x)
        if choice:
            next_char = np.random.choice(chars, p=F.softmax(output)[0].data.cpu().numpy())
        else:
            next_char_idx = output.topk(1)[1].data[0][0]
            next_char = chars[next_char_idx]
        # return next_char
        seed_string = seed_string + next_char
    print(seed_string)
#tmp = print_example(choice=False)
tmp = print_example(iters=40, choice=False)
tmp

ethics is a basic foundation of all thatéééééééééééééééééééééééééééééééééééééééé


# Samples

In [14]:
def get_random_sample_string(length=41):
    sample_place = random.randint(0, len(text)-length-1)
    sample = text[sample_place:sample_place+length]
    return sample
get_random_sample_string()

'progress must _necessarily_ take place: b'

In [15]:
sample_size = 50000

sample_sentence_size = 40
sample_data = torch.zeros(sample_size, sample_sentence_size, len(chars)) # .cuda()
#sample_target = torch.zeros((sample_size, sample_sentence_size, 1), torch.LongTensor)
sample_target = torch.LongTensor(sample_size, sample_sentence_size, 1).zero_() # .cuda()
for i in range(sample_size):
    sample = get_random_sample_string(sample_sentence_size+1)
    sample = [char_indices[c] for c in sample]
    for j in range(sample_sentence_size):
        sample_data[i][j][sample[j]] = 1
        sample_target[i][j] = sample[j+1]

In [16]:
i = 5
j = 10
sample_data[i].topk(1)[1][j][0] == sample_target[i][j-1][0]
''.join([chars[c[0]] for c in sample_data[i].topk(1)[1]])

True

'riority: whereas here, no less than in t'

In [17]:
clip_parameters_value = 0.25

def train_single(optimizer, loss_function, use_teacher_forcing):
    model.zero_grad()
    loss = 0
    
    sample_i = random.randint(0, len(sample_data)-1)
    x = sample_data[sample_i]
    y = sample_target[sample_i]
    
    input_char = x[0]
    target = Variable(y).cuda()
    
    output_line = []
    hidden = model.init_hidden()
    
    for i in range(len(x)):
        if use_teacher_forcing:
            input_char = x[i] # replace input with right target
        
        input_char = Variable(input_char).cuda().view(1,1,-1)
        output, hidden = model(input_char, hidden)
        loss += loss_function(output, target[i])

        char_idx = output.topk(1)[1].data[0][0]
        output_line.append(chars[char_idx])
        
        input_char = torch.zeros(1,1,len(chars))
        input_char[0,0,char_idx] = 1
            
    if clip_parameters_value:
        torch.nn.utils.clip_grad_norm(model.parameters(), clip_parameters_value)
    
    loss.backward()
    optimizer.step()

    return ''.join(output_line), loss.data[0] / len(sample)

#tmp = get_random_sample(20)
train_single(torch.optim.Adam(model.parameters(), lr=0.0001), nn.NLLLoss(), use_teacher_forcing=False)
train_single(torch.optim.Adam(model.parameters(), lr=0.0001), nn.NLLLoss(), use_teacher_forcing=True)
#tmp = train_single(tmp, torch.optim.Adam(model.parameters(), lr=0.0001), nn.NLLLoss(), use_teacher_forcing=True)

#tmp.topk(1)[1].data[0][0]

('""ggcggc99999999999999999999999999999999', 3.9511882502858233)

('gégg9gg9cé9cgcgcécé9c"g""""gggc9ccccé9cc', 3.9489061309070124)

In [18]:
def train_iterations(n_iters=100000, lr=0.001, teacher_forcing_ratio=0.5,
                     print_every=10000, plot_every=1000):
    global optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    start = time.time()
    model.train()

    current_loss = 0
    current_loss_iter = 0
    
    for iteration in range(1, n_iters + 1):
        model_training.iterations += 1
        
        use_teacher_forcing = random.random() < teacher_forcing_ratio
        
        result, loss = train_single(optimizer=optimizer, loss_function=nn.NLLLoss(),
                                    use_teacher_forcing=use_teacher_forcing)
        
        current_loss += loss
        current_loss_iter += 1

        # Print iter number, loss, name and guess
        if iteration % print_every == 0:
            teacher_forcing_str = ""
            if use_teacher_forcing:
                teacher_forcing_str = "(forcing)"
            #correct = '✓' if result == s_string else "✗: {}".format(s_string)
            
            print("{:>6d} {:>4.0%} ({:>8}) {:>7.3f}   | {:>6.2f}: {} {}".format(
                      model_training.iterations, iteration/n_iters, time_since(start),
                      current_loss/current_loss_iter, loss, result, teacher_forcing_str))

        # Add current loss avg to list of losses
        if iteration % plot_every == 0:
            model_training.losses.append(current_loss / plot_every)
            model_training.learning_rates.append(lr)
            current_loss = 0
            current_loss_iter = 0
            
        #if model_training.iterations % 50000 == 0 or model_training.iterations == 10:
        #    model_training.save_models()
        #    acc = test_model_accuracy(encoder_rnn, test_model_single_sample)
        #    model_training.accuracy.append(acc)
    
    # test_model_accuracy(model, n_sample=10000)

In [19]:
from pytorch_utils_oh_3 import ModelTraining
MODEL_SAVE_PATH = 'char_rnn_fast_ai_testing_2'

Pytorch utils oh: pytorch_utils_oh_3.py
Pytorch: 0.2.0_4


In [20]:
model_training = ModelTraining(MODEL_SAVE_PATH, [model])

Save path: data/models/char_rnn_fast_ai_testing_2


In [21]:
train_iterations(n_iters=10, print_every=5)

     5  50% (   0m 0s)   3.871   |   3.77: t                                        
    10 100% (   0m 0s)   3.602   |   3.05: t                     e   ee         e   (forcing)


### WITHOUT TEACHER FORCING

In [22]:
model = CharRNN(len(chars), 512).cuda()
model_training = ModelTraining(MODEL_SAVE_PATH, [model])
lr = 0.0001

Save path: data/models/char_rnn_fast_ai_testing_2


In [23]:
train_iterations(n_iters=10000, print_every=1000, teacher_forcing_ratio=0, lr=lr)

  1000  10% (  0m 59s)   3.054   |   3.13:                                          
  2000  20% (   2m 1s)   2.982   |   2.90:                                          
  3000  30% (  2m 58s)   2.968   |   3.03:                                          
  4000  40% (  3m 55s)   2.960   |   2.96:                                          
  5000  50% (  4m 53s)   2.956   |   2.62:                                          
  6000  60% (  5m 51s)   2.943   |   2.97:                                          
  7000  70% (  6m 48s)   2.958   |   3.02:                                          
  8000  80% (  7m 45s)   2.952   |   2.87:                                          
  9000  90% (  8m 42s)   2.957   |   2.98:                                          
 10000 100% (  9m 39s)   2.951   |   3.37:                                          


In [24]:
print_example()

ethics is a basic foundation of all thatnm nse gs
e-ptlsr  t,nfsoaepaaedstdcet hentsat e
niforoe ensnoohysr d
eatnsssioi ec ait drvx oounufseudra fom  loneihsu  t n encie r  va ue gehrwillicmoda "tisinoeychletre
e  uoeto ste hoe utiheooh to  dil d  usehinyld a"detd,re cthnn fnt eo
e "nphenphgacpsp    iedl a msfsn
=tuyhanreot r
hucer  oarm uanem ttn iacsl vlw


In [25]:
print_example()

ethics is a basic foundation of all thattrmlnu els . hhea llytesteasa tt s  nd oiomeutth t e r not  aote ssm neteeehchiugis - pfa vel noihr  ni   tae j u anlstuye ayu   ovdtb  va
 ts
"weeneheise atus"aeioctnlowi sce o ts   yaop "gyh  ana o
o lihdtlefe  -upnw od h
reast
sew    em eodeso rp .l oke 
 oddrao itse   eiasitoalorettnemnmente

e xo tses  ,ft l  eoet


In [26]:
train_iterations(n_iters=10000, print_every=1000, teacher_forcing_ratio=0, lr=lr*5)

 11000  10% (  0m 57s)   2.981   |   2.93:                                          
 12000  20% (  1m 54s)   2.971   |   3.14: an                                       
 13000  30% (  2m 51s)   2.967   |   2.86: n                                        
 14000  40% (  3m 48s)   2.965   |   2.89:  th                                      
 15000  50% (  4m 45s)   2.956   |   3.08:  t                                       
 16000  60% (  5m 42s)   2.962   |   3.07: to                                       
 17000  70% (  6m 39s)   2.949   |   2.92: he                                       
 18000  80% (  7m 37s)   2.956   |   3.02: ao                                       
 19000  90% (  8m 34s)   2.952   |   3.56: he                                       
 20000 100% (  9m 31s)   2.948   |   3.06:                                          


In [27]:
print_example()

ethics is a basic foundation of all thats atnscnoi outnl oc   en ,  e esah
eentoodiiaca eu
evoepoelf tee,   r  ect
, , heldr ctainasveueetdel  h segeitimenecm
aocinlr  s rs,,t  b
ss  t he natrutsy s  enr , leoein veor oseptn,   eondgs  . , or  t   oasacn whceo ns n l. erslu ances opfceireashes   er onl of  cereneun, eeeti , of  ssed lrt , 
lt-c  rtotfehaofun


In [28]:
print_example()

ethics is a basic foundation of all thatand, eeonos
 t e  ido plelohio upee n ttii sseanedn  sonens tlpsg es feh  
 veianes aheieite llhindot  nlheas e aveeltotsf ee p we,
ptnwa e lleborhe ersheesu ruhtensi whs  ua an  imsf cannthifce e ea hminneu e s,herussv eh eeritrlel
ip orteb r lotebnn, o  uinn e fd-= necflr .slgeomly  e  tuns e ntuof n runtoume o nveon


In [29]:
train_iterations(n_iters=10000, print_every=1000, teacher_forcing_ratio=0, lr=lr)

 21000  10% (  0m 57s)   2.933   |   2.94: n                                        
 22000  20% (  1m 54s)   2.941   |   2.84: eee                                      
 23000  30% (  2m 51s)   2.931   |   2.90: he                                       
 24000  40% (  3m 48s)   2.937   |   2.82: e                                        
 25000  50% (  4m 46s)   2.936   |   3.01: th                                       
 26000  60% (  5m 43s)   2.923   |   2.97: e                                        
 27000  70% (  6m 41s)   2.946   |   2.72:                                          
 28000  80% (  7m 39s)   2.934   |   3.00: n                                        
 29000  90% (  8m 36s)   2.929   |   2.93: he                                       
 30000 100% (  9m 33s)   2.936   |   3.12: e e                                      


In [30]:
print_example()

ethics is a basic foundation of all thatsnea rse f esu e aatachstseteopd -ah e tamlltouiththin ha a 
-  r,eos,  ae d   sel  otr eds ndfopndepret rsthit  -ule estaicscelieo    yinsre n stretuaieusyd mof r aiapdn tl ino  erosan r l,t, lthca
ttltpid dericmr aollt
 
a,s s nstirif  gss it uoem is mitlcidahehotlete ryy  mor lhel we  rm   h tat
e eof     i ekh  ndh


In [31]:
print_example()

ethics is a basic foundation of all thatr  w; the  rn sn si e 
 -ent
e dvn fptr mos ts ros  deate ie i antte in tt lil gacilr th hes va, odsty olaiunsws
 o r  a, theeue mstibss i 
ys gt d
iagnernpe-
ofrleaha r ruo  hrss ten  t  yis ot e
he esr  erovpis s theceste o mtemadelar ee ti ob s m h t crdeotro g  ell .ewholnl st des t he  are cs  ce fe ereegdoauatreo


In [32]:
train_iterations(n_iters=10000, print_every=1000, teacher_forcing_ratio=0, lr=lr)

 31000  10% (  0m 56s)   2.933   |   2.83: the                                      
 32000  20% (  1m 52s)   2.933   |   2.92: the                                      
 33000  30% (  2m 48s)   2.928   |   3.79: he                                       
 34000  40% (  3m 45s)   2.929   |   3.04:   o                                      
 35000  50% (  4m 41s)   2.929   |   2.96:                                          
 36000  60% (  5m 37s)   2.938   |   2.83: th                                       
 37000  70% (  6m 33s)   2.935   |   3.25: tn                                       
 38000  80% (  7m 30s)   2.930   |   2.84:  an                                      
 39000  90% (  8m 26s)   2.929   |   2.85: e                                        
 40000 100% (  9m 23s)   2.929   |   2.98: he                                       


In [33]:
print_example()

ethics is a basic foundation of all that, hit sa iwan
 hei, ei st)intt   ioffisiedo i  tad nn miseeanig  eneelisiln  ricssceth of  a uns r ndinof  e t to lias omisnnpn 
deosa hh poirineboors
nd t ror agoosn-s r g bees fe instmes, tiosfpfof hearer co, ne hit ng e
sactai iotrou ss ua,e s f tsineirmcath in
ssoed,  hde 
poerir , hetileisttun e e hel -ysa -s heeo


In [34]:
print_example()

ethics is a basic foundation of all thatienl hipol s  e"re einlind oneg b aed
bueore hh eirisarut t ce s ninnte  y,  diswof g o w  i nieh tos oue n  reen yn-eo n  n utretaos poen .is fmiaoorctne  ir s  
lihellway st sts  t 8it ,  ulethveugrw  ctyiee inserea, rns  aall,te gscoith d ste d  aanimst w ts e,   lheeismil
our
 ssall  ress o nrigge td alee,r  at nui


In [35]:
train_iterations(n_iters=10000, print_every=1000, teacher_forcing_ratio=0, lr=lr)

 41000  10% (  0m 56s)   2.932   |   3.06: re                                       
 42000  20% (  1m 53s)   2.930   |   2.81: e e                                      
 43000  30% (  2m 49s)   2.929   |   2.87:  th                                      
 44000  40% (  3m 46s)   2.927   |   2.84: th                                       
 45000  50% (  4m 43s)   2.923   |   2.96: th                                       
 46000  60% (  5m 40s)   2.937   |   2.78: n                                        
 47000  70% (  6m 37s)   2.936   |   3.04: n                                        
 48000  80% (  7m 33s)   2.932   |   2.77: re                                       
 49000  90% (  8m 30s)   2.934   |   2.95:                                          
 50000 100% (  9m 26s)   2.930   |   2.96: n                                        


In [36]:
print_example()

ethics is a basic foundation of all thate  f , dch gon  se   ereerin le a  eount nicrcess  oaovautgiinounc  a tohi gaseda naca oetas tir"he 
 ar" rthow ft higr e c-ithio rion ed h tbeepiei  d clye whce    ns  . s  souebe atos stoeaicieseevotem ies n r iedgacime ng g
nshehed  rordh s oa ieice slc ealeonmetiege t   rm is e s e cnad tarewin sed
 ed  aru  ie mes


In [37]:
print_example()

ethics is a basic foundation of all thatae  ve co h  tsita  ebsitie ,  aowine s hxtred wgh ini aldeu , m
n lnl    as sphsstd hoit b ule tk  gciatca h
fii beconl    dogunbidy tr, eriveboutmrr eh t refood nri eisorieh,
ofn,   mt tie  enitouior iinteofarrs unanl la oinli sn nd  n hm
ly  uost s ahisle 
hpreciglereglid o s, t yceings seenisgb had he n igr 
cancte


In [38]:
train_iterations(n_iters=50000, print_every=10000, teacher_forcing_ratio=0, lr=lr)

 60000  20% (  9m 23s)   2.924   |   3.21:   a                                      
 70000  40% ( 18m 45s)   2.928   |   2.73:  to                                      
 80000  60% (  28m 5s)   2.924   |   2.72:  ah                                      
 90000  80% ( 37m 26s)   2.928   |   3.27: e                                        
100000 100% ( 46m 48s)   2.924   |   2.96:  rh                                      


In [39]:
print_example()

ethics is a basic foundation of all thath aenn theeght o a n-es   , eis erosm ty 
ey 
veeke ofsy dig t  ed  ofent neeg mo a itmrpe,
 oneil dosannli ent mete rtt d  itsk  hseresem  t 
alooerenwiteon, ,e    ison rthen s lio l phhss  ass  mensef  v  io so" ltaieceateb rf oe eed-t o
ileto 
t is, eld netsicnyi eidan  anhedgeyerr   s pea pr
ryss  ceearur  l, ;att 


In [40]:
print_example()

ethics is a basic foundation of all that  acfse n  s toered,t :
gedeareoiemlg
t, h
tmr  o es rsusoveo tampech msa, nt a (e of on oo 
sta   ine atisf oa i anbsfet are,
s i cemim tus
le nd eateunesusr hemoy .
 s  nt y lr n linde
et s  eiscevie lneeo ,  ta er  gi  yo sy inewe a t  legilyko seo n de; nuto  etse usls eeseceamistits n gh hendin  2t ne f itr .!  s 


In [41]:
print_example()

ethics is a basic foundation of all that s, g eesptftued afio iscamsrtecty r pente t eicivys f ns rteole ci,ous htes ue, erin,
t 'e s ess   

atepeeecicercetir 
isecod
tee nweenpe fios te heepoarpaulo 
dserwecoreisiog
h nt  arpall r ie t thae
r ledsteit, lla mco, 
esd toeits, rhte ir  ey peeay usld s  eebr ,a 
 fh sair s  toncillinanry s e  s e rhephisserhie


In [42]:
train_iterations(n_iters=50000, print_every=10000, teacher_forcing_ratio=0, lr=lr)

110000  20% (  9m 18s)   2.937   |   2.99:   o                                      
120000  40% ( 18m 38s)   2.928   |   3.26:   oe                                     
130000  60% ( 27m 58s)   2.926   |   2.73: th                                       
140000  80% ( 37m 18s)   2.926   |   2.89: th                                       
150000 100% ( 46m 39s)   2.928   |   2.74: th                                       


In [43]:
print_example()

ethics is a basic foundation of all that , f
raa ras d ty onyt blhalit dal
de n or fdene  t heo n  cevle dtar ,  e  nmsesunst yd g  ieuanitd rglef lico ue e, ethlieh
gofeiphtr alorc ilde ehlall t e ty allerahreaioe
 rt e s  hef mn   d7ie  n  y   poiy heid rseeeets brehys  cnec, aldraheol  deods e ssca meo g   rloa tdait   o ofucceth l
ledse eense(n. ta rs ie


In [44]:
print_example()

ethics is a basic foundation of all that wone ndty ed inedd,  ustenye,, h, sen seaccoofe  ocbchteo ot thwemus,
ro ng oan thet end s oyt  eshe ed , atyn r  e is farne ico
fa rgescitikhonatid t ingf d, rn, s a d  as  -ttr se
sa santodn isbet  icthe sy yss t ; iielishrte uley. bueda eeend  cse
g al  s t e  use tmsoen   e, oa e theec  n h  e henos diorryenetanve


In [45]:
print_example()

ethics is a basic foundation of all thatnansse t, rrt lee in , h bt,  iisar  
ntrypre  ivinon ie- welitl  lles.  sulet ukacn  y y 
 s whah, su ihsf t a keo 1s  en e ys fi y  n  wgrikme  lviroot  onser , aoyssosn og et  oterall eedamy  ofenaetro had  so   onndmeiadennha ity. ubtor me  ebyiof ,at d
ddre ngi, oieqiiroe on rlucaene, t,, ie  enn ho  fit  nane"eno


In [46]:
model_no_teach = [model, model_training]

### WITH TEACHER FORCING

In [47]:
model = CharRNN(len(chars), 512).cuda()
model_training = ModelTraining(MODEL_SAVE_PATH, [model])
lr = 0.0001

Save path: data/models/char_rnn_fast_ai_testing_2


In [48]:
train_iterations(n_iters=10000, print_every=1000, teacher_forcing_ratio=0.5, lr=lr)

  1000  10% (  0m 56s)   3.040   |   3.07:                                          
  2000  20% (  1m 53s)   2.928   |   2.69: e          e         ee  e e    n        (forcing)
  3000  30% (  2m 49s)   2.829   |   2.96: the   on  on  on  on  on  on  on  on  on 
  4000  40% (  3m 46s)   2.781   |   3.12: the e  e  e  e  e  e  e  e  e  e  e  e   
  5000  50% (  4m 42s)   2.772   |   2.43: tenhen  ene  e ae ens  te n tg   an  te  (forcing)
  6000  60% (  5m 39s)   2.746   |   2.48: n  ane  ie teen en ao     t reahe  e n   (forcing)
  7000  70% (  6m 36s)   2.722   |   2.47: e   t thet th  an tnd  e etn   ne e thet (forcing)
  8000  80% (  7m 33s)   2.717   |   2.42:   n   eahe       r eaf the   enn  n th   (forcing)
  9000  90% (  8m 29s)   2.671   |   2.05: nete enn an the  e eendt  an ahe    ene  (forcing)
 10000 100% (  9m 26s)   2.653   |   2.43: e  re tn   ahreeng o  the  hen en ao   n (forcing)


In [49]:
print_example()

ethics is a basic foundation of all thath
wallg batt cale anrots
aonde ioco aivedinh callat copek cedcacferore ind 
enpas ingons  in  at  tthely
hee is
torpaasithisnenredaccnces ion rrecid, dopireanebcs, or  afmas anrer es io the  in t memend
stythimase tinm.zty h  es, theha-ingf,ery th manathy seaclust tamt of"uw enitht be)-itn cime torweabi;nsitan w  ad is


In [50]:
print_example()

ethics is a basic foundation of all thatetcoesefrsatewhand themeso thrimng(intttustpinnt othis:yof ho  ung oust, su heolingomathace"-ttat or ame iis maver os eim rale,  aves yheg phet
eon
,-
nh end et ounelatstind thse tinnt w nareiqs, si ndewintoso gee the t redhlics ie aid yd
inthehstivd endhatle d an for sheoigpeid fen audenosving in : eb wtofpals, ouek
a


In [51]:
train_iterations(n_iters=10000, print_every=1000, teacher_forcing_ratio=0.5, lr=lr*5)

 11000  10% (  0m 56s)   2.686   |   2.11: n  tnen nn  iheth ih tneerenn e tn the   (forcing)
 12000  20% (  1m 53s)   2.633   |   2.19: eth tndere tor  tetns ooitetyoow  aot to (forcing)
 13000  30% (  2m 49s)   2.542   |   2.04: e   tf tf  s af ttot re  ahet teirsos  t (forcing)
 14000  40% (  3m 45s)   2.568   |   2.41: eue ndioslere  an eafer sg toi  aot ihen (forcing)
 15000  50% (  4m 42s)   2.546   |   2.00: e aoeuith l   andersahe pnsoesdte of trr (forcing)
 16000  60% (  5m 38s)   2.502   |   1.65: heng  ah e tesieas of mheth  tet ihit in (forcing)
 17000  70% (  6m 34s)   2.464   |   3.15:  the                                     
 18000  80% (  7m 31s)   2.460   |   3.13: itis and the                             
 19000  90% (  8m 28s)   2.413   |   3.10: e                                        
 20000 100% (  9m 24s)   2.425   |   3.55: e a                                      


In [52]:
print_example()

ethics is a basic foundation of all that hes aul the rpasios, bere m8-thiske to of thesrechs: therechenge y wele
thereking the
ut os r selfecred. in
sellems, as ascessinrust
wion"--whe hewer, in
owe thes chele empors hith oor mare
thimmed, as ofd barkictive,
which moye pruaseror baldes of gveres goig thoughtre thing!


4. this oon wi5h that is a constomy and


In [53]:
print_example()

ethics is a basic foundation of all that awherestation the ecolitute, and whel bus  e wainratirn wnowherse--hull--if to theol mednatres efreding of like must in and thus ictine thearthoubem--hy these thera hage belies;"
in  he kiphe the freetsuce earers as realat is o" . just, which pose felf--remattous shilks and that the resimevilaltedoby, wave not asmavee


In [54]:
train_iterations(n_iters=10000, print_every=1000, teacher_forcing_ratio=0.3, lr=lr)

 21000  10% (  0m 56s)   2.621   |   1.55: etity aashert   --tn is ahe eonet ovenin (forcing)
 22000  20% (  1m 53s)   2.583   |   2.93: e the                                    
 23000  30% (  2m 49s)   2.637   |   3.18: e                                        
 24000  40% (  3m 46s)   2.585   |   2.74: he                                       
 25000  50% (  4m 42s)   2.623   |   1.59:   n tot eng thet ies tonse  tonth tan ni (forcing)
 26000  60% (  5m 39s)   2.594   |   2.79: the                                      
 27000  70% (  6m 36s)   2.616   |   3.01: n                                        
 28000  80% (  7m 33s)   2.542   |   2.97:  reall to                                
 29000  90% (  8m 29s)   2.584   |   1.59: ne ohet ifer nnce ais be n aort areceice (forcing)
 30000 100% (  9m 26s)   2.570   |   1.72: hreon  afr aifeteteon  afr aeilediny  
n (forcing)


In [55]:
print_example()

ethics is a basic foundation of all that andvelves ppicianmesthe stopt, redition that an. are jeing an the urfuchrevalis? instretd,
and tiemtis "bud chalay destinctic is yse bo dais,
resail instirld, with every
to howher--in atd most altueder of there ane hisself
almost
s
in of in. in the to this to the beverour every -endy stalles, hapstation (fich, letout 


In [56]:
print_example()

ethics is a basic foundation of all that serfoundsy er.---s chase interunge pracfulaited becro witlome
muth diffan intgates of recandres and al our fon the relngrous of
him in the his conng(ingoud antamblez-digmation, by mereat him utorialite the ranufess felte an free hast--gonden, and can invelttence on reans of dinderen amable
berpiceral the helkgrelves o


In [57]:
train_iterations(n_iters=10000, print_every=1000, teacher_forcing_ratio=0.2, lr=lr)

 31000  10% (  0m 56s)   2.698   |   1.76:  tonlrre af tistreati tf ng th testrrii  (forcing)
 32000  20% (  1m 53s)   2.694   |   2.97:  to the                                  
 33000  30% (  2m 49s)   2.686   |   2.81: e st a                                   
 34000  40% (  3m 46s)   2.693   |   2.85: th                                       
 35000  50% (  4m 43s)   2.695   |   2.71: he                                       
 36000  60% (  5m 39s)   2.712   |   2.87:                                          
 37000  70% (  6m 35s)   2.689   |   2.79: th                                       
 38000  80% (  7m 32s)   2.679   |   2.91: e                                        
 39000  90% (  8m 28s)   2.703   |   3.19: nd                                       
 40000 100% (  9m 25s)   2.702   |   1.51: sllthi  tor the sone tf teesenteng tisse (forcing)


In [58]:
print_example()

ethics is a basic foundation of all that rend--if,
would ashemony relagion, and casse he dlessoas shata to to semplay wat very coss mankeal syes of ourged hement, or aversong ass aelirety
ene's of ever the have way prove oneys i te fortupod he han coutredirar lisely of conthile his odrylise, themecolackes, more anwind verother wored. nes he inst"uct of hor o


In [59]:
print_example()

ethics is a basic foundation of all that be lirenc some of all the bass, condry,
and the consci acling his own of the
actom
that mouald relager of at nelle imitin, not onus in thour, led past? in
whated; hy cempory--sim; to atobiticals in axpriedsity.d were here ha  fould sianes (somethat lofe the hisher mage it sf wiral masure but deding to gros.
prasitiesa


In [60]:
train_iterations(n_iters=10000, print_every=1000, teacher_forcing_ratio=0.1, lr=lr)

 41000  10% (  0m 55s)   2.817   |   1.57: tot te oseuinte on the seatientid torte  (forcing)
 42000  20% (  1m 51s)   2.816   |   1.71: st   e  et  eet lost an erpnian treer th (forcing)
 43000  30% (  2m 47s)   2.809   |   3.12:  eaine                                   
 44000  40% (  3m 43s)   2.835   |   2.98: e te                                     
 45000  50% (  4m 39s)   2.799   |   2.95: nd                                       
 46000  60% (  5m 35s)   2.839   |   2.84: et                                       
 47000  70% (  6m 31s)   2.801   |   2.83: e ee                                     
 48000  80% (  7m 27s)   2.801   |   3.04:  ea                                      
 49000  90% (  8m 23s)   2.811   |   3.14: n                                        
 50000 100% (  9m 19s)   2.832   |   2.76: he                                       


In [61]:
print_example()

ethics is a basic foundation of all that
so imans-ant and-nitfured bat atisel him.

284. great endases now resuges soun-saduse curforurnceming we spact. not meraided with
ceatain perssiin priphing nothers deed tints! in feal thates and theis mataiving, pristimanly be,y
has apbees for hewith tieses acest the altoreading thein syund would blan gradber, smartie


In [62]:
print_example()

ethics is a basic foundation of all that ot the inrepaons of
t is suce a seepant. a
matake: to the thachthen betoregon. dagnestos banging falicobyity, of
which the ever of pliotians, is anf ict as moralizes, be all a that who have bempertory is wo
ld all as over? he
douch in the toustt forger but may no to commuly. nhaures ordee is the corrament." an the to 


In [63]:
train_iterations(n_iters=50000, print_every=10000, teacher_forcing_ratio=0, lr=lr)

 60000  20% (  9m 24s)   2.926   |   2.96: e se                                     
 70000  40% ( 18m 48s)   2.930   |   3.18:                                          
 80000  60% ( 28m 12s)   2.926   |   2.81: he                                       
 90000  80% ( 37m 36s)   2.928   |   3.02:                                          
100000 100% ( 46m 58s)   2.927   |   2.86:                                          


In [64]:
print_example()

ethics is a basic foundation of all that a foshige more horthared undauthoug centas priape nacious and a deto ee humbel in?;
tleo teghing rek is y iar proan forrasion a keuader of the de h in
prsed in sectetsos.
213..-ho
 too beribed us eaved to thing au wongrnd astaake such-anari5),
ampious desplbeings tole, noblenselver itgherly hinden ir de eoire the ciem


In [65]:
print_example()

ethics is a basic foundation of all that ther igse had and hildich wh e gehnar bed
dols live, he poprebo n toherquisply reconjeys--eele prejure he hid andi  malofit
it if ory reaverte polus be possionisecluseoft..
2]verelai-g ent, woranious--to trken instluth ofterviisuce.--bus pregssaid" ir that of gron the  pee; prods we pricamand anmonth"ar of the godl in


In [66]:
print_example()

ethics is a basic foundation of all that thengy
as picioupeermother ngouble befo are wemprubudisi_ally foresion eed an taeipass ale the is astert onetldonght
becoeitling, he hag=--ofayy mer;
all a perictianiy iepoomansy, (he veryother leom, thr irct sim er for enes ofciy of thattiinelsertistndandinicurt,
os.. t165.
iei tier  tdetneert ew hectoceat do in opes


In [67]:
train_iterations(n_iters=50000, print_every=10000, teacher_forcing_ratio=0, lr=lr)

110000  20% (  9m 22s)   2.923   |   2.90: n                                        
120000  40% ( 18m 44s)   2.933   |   2.85:                                          
130000  60% (  28m 7s)   2.924   |   3.26: re                                       
140000  80% ( 37m 33s)   2.923   |   2.95: the  e                                   
150000 100% ( 46m 57s)   2.930   |   2.79: n                                        


In [68]:
print_example()

ethics is a basic foundation of all that er cheferbig
than--acreors onls snculi,
beschen,
bum as, mselferer permscoriit exaly!--n13 thaull gned part has ofter  head on  spespo, had seace ly wastuch toble d
beiour
ore to ele n ?
phaie of tede seldhing hid
vanuasel whund , as n: i incuanticat cincimen efel," as
cort anething fang treh is prongoi th toelw lfode


In [69]:
print_example()

ethics is a basic foundation of all thato gceifags, tys efe dixtaitly
whrmecerman le. theis thes erthenc apfer  cpoth bee
meytth, and
ging-worghthen we leniun wighs d y arosival,.=-hhur sery ond, noe  sadoand a tiner allofd takeooreulesel womane: to chrnefnin werm. anl hempo, uper uama in, we fry beniracomy"'hy whece ofas ampoweth wandne fregith nerusinyt na


In [70]:
print_example()

ethics is a basic foundation of all thatere?

16horovigian o t raage ofit nothen anlishell, ho gleglest--by one, th tiger peou4, do stepoe omtillle
f,"g
to moet pessicutee
grean of cell'gercon in whwtr almogading te dom
even hiawer re ir then o ntble sould,"-he mevel thlys in man for and lence sell."dimper umome fa sulftheveraby,
and t eurhen ferscegs and si
